In [1]:
import pandas as pd
import textract
import glob
import re
from nltk.corpus import stopwords

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jasleen.bhatia@fiserv.com/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
default_files = ['doc', 'docx', 'pdf','DOC', 'DOCX', 'PDF']

In [4]:
def resume2text(file_path):
    content = textract.process(file_path)
    content = str(content, 'utf-8')
    lines = content.split('\n')
    paragraph = ''
    for line in lines:
        if line != '':
            paragraph += line + '\n'
    return paragraph

In [34]:
def extract_keywords(text):
  rules = [
            {r'>\s+': u'>'},  
            {r'\s+': u' '},  
            {r'\s*<br\s*/?>\s*': u'\n'}, 
            {r'</(div)\s*>\s*': u'\n'},  
            {r'</(p|h\d)\s*>\s*': u'\n\n'},  
            {r'<head>.*<\s*(/head|body)[^>]*>': u''},  
            {r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'}, 
            {r'[ \t]*<[^<]*?/?>': u''}, 
            {r'^\s+': u''}  
          ]
  for rule in rules:
    for (k, v) in rule.items():
      regex = re.compile(k)
      text = regex.sub(v, text)
    text = text.lower()
    text = re.sub("\s\s+", " ", text) 
    text = re.sub('[\d]', '', text)
    text= ' '.join(text.split())
    stop_words = stopwords.words('english')
    text = ' '.join([word for word in text.split() if word not in (stop_words)])
  return text


In [6]:
def read_resumes(label,path_to_folder, file_types=default_files):
    texted_files = []
    file_names = []
    labels=[]
    for file_type in file_types:
        file_path = path_to_folder
        file_path += '/*.' + file_type
        files = glob.glob(file_path)
        for file in files:
            try:
                text = resume2text(file)
            except:
                print('Fail to parse ' + file)
                continue
            text = text.strip()
            if len(text) == 0:
                print('Fail to parse2 ' + file)
            else:
                texted_files.append(text)
                labels.append(label)
                file_names.append(file)
                listt=[]
                for texted in texted_files:
                        keywords = extract_keywords(texted)
                        
                        listt.append(keywords)
                d1={"Text":listt,"Labels":labels}
                df=pd.DataFrame(d1)
    return df,file_names 
